In [1]:
%pip install tensorflow -q

import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical


Note: you may need to restart the kernel to use updated packages.


In [4]:
from pathlib import Path
import numpy as np

# Detect project root (handles running from /notebooks or root)
CWD = Path().resolve()
if CWD.name == "notebooks":
    PROJECT_ROOT = CWD.parent
else:
    PROJECT_ROOT = CWD

print("CWD:", CWD)
print("Assumed PROJECT_ROOT:", PROJECT_ROOT)

# Primary (correct) expected location
primary_npz = PROJECT_ROOT / "data" / "processed" / "classification_data.npz"
# Old (wrong) location under notebooks
legacy_npz = PROJECT_ROOT / "notebooks" / "data" / "processed" / "classification_data.npz"

print("\nLooking for classification_data.npz in:")
print("1:", primary_npz, "exists:", primary_npz.exists())
print("2:", legacy_npz, "exists:", legacy_npz.exists())

# Decide which path to use
if primary_npz.exists():
    data_path = primary_npz
elif legacy_npz.exists():
    print("\n⚠ Using legacy path inside notebooks (better to move file later).")
    data_path = legacy_npz
else:
    raise FileNotFoundError(
        f"❌ classification_data.npz not found.\n"
        f"Expected at:\n  {primary_npz}\n  or (legacy) {legacy_npz}\n"
        f"Please run 02_preprocessing_cnn.ipynb first."
    )

print("\n✅ Loading:", data_path)

data = np.load(data_path)

X_train = data["X_train"]
y_train = data["y_train"]
X_val = data["X_val"]
y_val = data["y_val"]
X_test = data["X_test"]
y_test = data["y_test"]

print("\nShapes loaded:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:  ", X_val.shape,   "y_val:  ", y_val.shape)
print("X_test: ", X_test.shape,  "y_test: ", y_test.shape)

# Safety check: stop if empty
if X_train.size == 0 or X_val.size == 0 or X_test.size == 0:
    raise ValueError(
        "❌ Loaded arrays are EMPTY.\n"
        "This means 02_preprocessing_cnn.ipynb did not correctly load/save the images.\n"
        "Re-run 02 from top to bottom, then rerun this notebook."
    )

num_classes = len(np.unique(y_train))
input_shape = X_train.shape[1:]

print("\nNumber of classes:", num_classes)
print("Input shape:", input_shape)


CWD: F:\Aerial_Object_Classification_Detection\notebooks
Assumed PROJECT_ROOT: F:\Aerial_Object_Classification_Detection

Looking for classification_data.npz in:
1: F:\Aerial_Object_Classification_Detection\data\processed\classification_data.npz exists: True
2: F:\Aerial_Object_Classification_Detection\notebooks\data\processed\classification_data.npz exists: True

✅ Loading: F:\Aerial_Object_Classification_Detection\data\processed\classification_data.npz

Shapes loaded:
X_train: (2662, 224, 224, 3) y_train: (2662,)
X_val:   (442, 224, 224, 3) y_val:   (442,)
X_test:  (215, 224, 224, 3) y_test:  (215,)

Number of classes: 2
Input shape: (224, 224, 3)


In [5]:
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_val_cat = to_categorical(y_val, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

print("Example y_train (raw):", y_train[:5])
print("Example y_train_cat:", y_train_cat[:5])


Example y_train (raw): [0 0 0 0 0]
Example y_train_cat: [[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [6]:
input_shape = X_train.shape[1:]  # (224, 224, 3)

def build_custom_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),

        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),

        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),

        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),

        layers.Conv2D(256, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),

        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax")
    ])
    return model

model = build_custom_cnn(input_shape, num_classes)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 14, 14, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    12,845,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,236,162 (50.49 MB)

 Trainable params: 13,235,202 (50.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [7]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"]
)

print("Model compiled.")


Model compiled.


In [8]:
checkpoint_path = MODELS_DIR / "custom_cnn_best.h5"

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    filepath=str(checkpoint_path),
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

callbacks = [early_stopping, model_checkpoint]

print("Checkpoint will be saved to:", checkpoint_path.resolve())


Checkpoint will be saved to: F:\Aerial_Object_Classification_Detection\notebooks\models\classification\custom_cnn_best.h5


In [9]:
BATCH_SIZE = 32
EPOCHS = 30

history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6903 - loss: 1.2965
Epoch 1: val_loss improved from None to 0.71555, saving model to models\classification\custom_cnn_best.h5


84/84 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.7442 - loss: 1.0593 - val_accuracy: 0.4910 - val_loss: 0.7156
Epoch 2/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8854 - loss: 0.3541
Epoch 2: val_loss did not improve from 0.71555
84/84 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.8847 - loss: 0.3365 - val_accuracy: 0.4910 - val_loss: 1.0502
Epoch 3/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9250 - loss: 0.2091
Epoch 3: val_loss did not improve from 0.71555
84/84 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.9185 - loss: 0.2208 - val_accuracy: 0.5385 - val_loss: 0.9691
Epoch 4/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9402 - loss: 0.1604
Epoch 4: val_loss did not improve from 0.71555
84/84 ━━━━━━━━━━━━━━━━━━━━ 126s 2s/step - accuracy: 0.9399 - loss: 0.1552 - val_accuracy: 0.6991 - val_loss: 0.8866
Epoch 5/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9416 - loss: 0.1485
Epoch 5: val_loss did not improve from 0.71555
84/84 ━━━━━━━━━━

84/84 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.9681 - loss: 0.0870 - val_accuracy: 0.8054 - val_loss: 0.6908
Epoch 7/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9796 - loss: 0.0576
Epoch 7: val_loss did not improve from 0.69084
84/84 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.9752 - loss: 0.0664 - val_accuracy: 0.7986 - val_loss: 0.7630
Epoch 8/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9713 - loss: 0.0776
Epoch 8: val_loss improved from 0.69084 to 0.55009, saving model to models\classification\custom_cnn_best.h5


84/84 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.9677 - loss: 0.0881 - val_accuracy: 0.8575 - val_loss: 0.5501
Epoch 9/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9817 - loss: 0.0588
Epoch 9: val_loss did not improve from 0.55009
84/84 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.9778 - loss: 0.0664 - val_accuracy: 0.8303 - val_loss: 0.6871
Epoch 10/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9809 - loss: 0.0493
Epoch 10: val_loss did not improve from 0.55009
84/84 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.9838 - loss: 0.0467 - val_accuracy: 0.8643 - val_loss: 0.6283
Epoch 11/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9856 - loss: 0.0537
Epoch 11: val_loss did not improve from 0.55009
84/84 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.9842 - loss: 0.0483 - val_accuracy: 0.8507 - val_loss: 0.6707
Epoch 12/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9902 - loss: 0.0254
Epoch 12: val_loss did not improve from 0.55009
84/84 ━━━━